In [31]:
import torch 
import torch.nn as nn
import os
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import torchvision

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Hyper-parameters
sequence_length = 19
input_size = 128
num_classes = 4
batch_size = 64
num_epochs = 10
learning_rate = 0.001
n_samples = 764

In [39]:
x = torch.randn(n_samples, 1, sequence_length, input_size)
y = torch.tensor(np.random.choice([0,1,2,3,4], n_samples))

In [40]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.data = x
        self.label = y
        
    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        data = torch.tensor(self.data[idx], dtype=torch.float32)
        label = torch.tensor(self.label[idx], dtype=torch.int64)
        return data, label

In [41]:
idx = int(len(y)*0.8)
train_x = x[:idx]
test_x = x[idx:]
train_y = y[:idx]
test_y = y[idx:]

In [42]:
trainset = CustomDataset(train_x, train_y)
testset = CustomDataset(test_x, test_y)
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, drop_last=True)

In [43]:
#dataset shape 확인
trainset[0][0].shape

<ipython-input-40-2b9bfe8c4c65>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(self.data[idx], dtype=torch.float32)
<ipython-input-40-2b9bfe8c4c65>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.label[idx], dtype=torch.int64)


torch.Size([1, 18, 128])

In [44]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [83]:
# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=5):
        super(ResNet, self).__init__()
        self.conv = conv3x3(1, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, 16, layers[0], 2)
        self.layer2 = self.make_layer(block, 16, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 32, 64, layers[2], 2)
        self.layer4 = self.make_layer(block, 64, 128, layers[3], 2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(128, num_classes)

    def make_layer(self, block, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(in_channels, out_channels, stride, downsample))
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

model = ResNet(ResidualBlock, [3, 4, 6, 3], num_classes=5).to(device)

In [119]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)
# 1x1 convolution
def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, 
                     stride=stride, padding=0, bias=False)

# Residual block
class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, channels, stride=1, downsample=None, base_width=16):
        super(Bottleneck, self).__init__()
        width = int(channels * (base_width / 16.))
        self.conv1 = conv1x1(in_channels, width)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = conv3x3(width, width, stride)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = conv1x1(width, channels * self.expansion)
        self.bn3 = nn.BatchNorm2d(channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = self.bn3(out)
        
        if self.downsample:
            identity = self.downsample(x)
            
        out += identity
        out = self.relu(out)
        
        return out

In [124]:
class ResNet(nn.Module):
    def __init__(self,
                block,
                layers,
                num_classes,
                base_width):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.base_width = base_width
        self.conv1 = nn.Conv2d(1, self.in_channels, kernel_size=5, stride=2, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(128*block.expansion, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def _make_layer(self, block, channels, n_blocks, stride):
        downsample = None
        
        if stride != 1 or self.in_channels != channels*block.expansion:
            downsample = nn.Sequential(conv1x1(self.in_channels, channels*block.expansion, stride),
                                      nn.BatchNorm2d(channels*block.expansion))
        
        layers = list()
        layers.append(block(self.in_channels, channels, stride, downsample, self.base_width))
        
        self.in_channels = channels*block.expansion
        for _ in range(1, n_blocks):
            layers.append(block(self.in_channels, channels, base_width = self.base_width))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [131]:
model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes=5, base_width=16).to(device)

In [132]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [133]:
total_step = len(train_loader)
curr_lr = learning_rate

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 3 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

<ipython-input-40-2b9bfe8c4c65>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(self.data[idx], dtype=torch.float32)
<ipython-input-40-2b9bfe8c4c65>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.label[idx], dtype=torch.int64)


Epoch [1/10], Step [3/9] Loss: 1.7333
Epoch [1/10], Step [6/9] Loss: 1.7312
Epoch [1/10], Step [9/9] Loss: 1.6636
Epoch [2/10], Step [3/9] Loss: 1.5510
Epoch [2/10], Step [6/9] Loss: 1.6501
Epoch [2/10], Step [9/9] Loss: 1.4839
Epoch [3/10], Step [3/9] Loss: 1.3469
Epoch [3/10], Step [6/9] Loss: 1.2983
Epoch [3/10], Step [9/9] Loss: 1.4031
Epoch [4/10], Step [3/9] Loss: 1.1640
Epoch [4/10], Step [6/9] Loss: 1.1048
Epoch [4/10], Step [9/9] Loss: 1.2072
Epoch [5/10], Step [3/9] Loss: 0.9197
Epoch [5/10], Step [6/9] Loss: 0.8688
Epoch [5/10], Step [9/9] Loss: 1.1199
Epoch [6/10], Step [3/9] Loss: 0.7210
Epoch [6/10], Step [6/9] Loss: 0.8956
Epoch [6/10], Step [9/9] Loss: 0.7192
Epoch [7/10], Step [3/9] Loss: 0.6454
Epoch [7/10], Step [6/9] Loss: 0.9368
Epoch [7/10], Step [9/9] Loss: 0.8369
Epoch [8/10], Step [3/9] Loss: 0.4196
Epoch [8/10], Step [6/9] Loss: 0.7175
Epoch [8/10], Step [9/9] Loss: 0.8625
Epoch [9/10], Step [3/9] Loss: 0.5149
Epoch [9/10], Step [6/9] Loss: 0.6049
Epoch [9/10]

In [134]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Accuracy of the model on the test images: 15.625 %


<ipython-input-40-2b9bfe8c4c65>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.tensor(self.data[idx], dtype=torch.float32)
<ipython-input-40-2b9bfe8c4c65>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(self.label[idx], dtype=torch.int64)
